In [7]:
# from dotenv import load_dotenv
import os
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential


def main():
    global cog_endpoint
    global cog_key

    try:
        # Get Configuration Settings
#         load_dotenv()
        cog_endpoint = 'https://azml1226.cognitiveservices.azure.com/' #os.getenv('COG_SERVICE_ENDPOINT')
        key_vault_name =  'azkv1226' #os.getenv('KEY_VAULT')
        app_tenant = '41a43e50-dc8c-46d7-9085-d04ba2e69216' #os.getenv('TENANT_ID')
        app_id = 'bd99a80d-3041-4d14-83b7-204305f4aef6' # os.getenv('APP_ID')
        app_password = '9mT1ZKibcLgEGRgHVf3dnItA9uPLYz-poZ' #os.getenv('APP_PASSWORD')

        # Get cognitive services key from keyvault using the service principal credentials
        key_vault_uri = f"https://{key_vault_name}.vault.azure.net/"
        credential = ClientSecretCredential(app_tenant, app_id, app_password)
        keyvault_client = SecretClient(key_vault_uri, credential)
        secret_key = keyvault_client.get_secret("Cognitive-Services-Key")
        print(secret_key)
        cog_key = secret_key.value
# 'f054517558d249a58cc2368f1141fa3f'        
        print(cog_key)
        # Get user input (until they enter "quit")
        userText =''
        while userText.lower() != 'quit':
            userText = input('\nEnter some text ("quit" to stop)\n')
            if userText.lower() != 'quit':
                language = GetLanguage(userText)
                print('Language:', language)

    except Exception as ex:
        print(ex)

def GetLanguage(text):

    # Create client using endpoint and key
    credential = AzureKeyCredential(cog_key)
    client = TextAnalyticsClient(endpoint=cog_endpoint, credential=credential)

    # Call the service to get the detected language
    detectedLanguage = client.detect_language(documents = [text])[0]
    return detectedLanguage.primary_language.name


if __name__ == "__main__":
    main()

<KeyVaultSecret [https://azkv1226.vault.azure.net/secrets/Cognitive-Services-Key/4f461417476048bea3c7f45c1514ba39]>
f054517558d249a58cc2368f1141fa3f

Enter some text ("quit" to stop)
How are you
Language: English

Enter some text ("quit" to stop)
quit


In [ ]:
'''
作業紀錄
az ad sp create-for-rbac -n "api://ai-app" --role owner --scopes subscriptions/8ccb2476-cfb9-4ac4-8ac4-c43b345cf82f/resourceGroups/eastasia

Creating 'owner' role assignment under scope 'subscriptions/8ccb2476-cfb9-4ac4-8ac4-c43b345cf82f/resourceGroups/eastasia'


{
  "appId": "bd99a80d-3041-4d14-83b7-204305f4aef6",
  "displayName": "api://ai-app",
  "name": "bd99a80d-3041-4d14-83b7-204305f4aef6",
  "password": "9mT1ZKibcLgEGRgHVf3dnItA9uPLYz-poZ",
  "tenant": "41a43e50-dc8c-46d7-9085-d04ba2e69216"
}

az ad sp show --id bd99a80d-3041-4d14-83b7-204305f4aef6 --query objectId --out tsv

c:\python>az ad sp show --id bd99a80d-3041-4d14-83b7-204305f4aef6 --query objectId --out tsv
4bd9e31a-8582-4d19-a7bb-748d7c7a384b

az keyvault set-policy -n azkv1226 --object-id 4bd9e31a-8582-4d19-a7bb-748d7c7a384b --secret-permissions get list


c:\python>az keyvault set-policy -n azkv1226 --object-id 4bd9e31a-8582-4d19-a7bb-748d7c7a384b --secret-permissions get list
{
  "id": "/subscriptions/8ccb2476-cfb9-4ac4-8ac4-c43b345cf82f/resourceGroups/eastasia/providers/Microsoft.KeyVault/vaults/azkv1226",
  "location": "eastasia",
  "name": "azkv1226",
  "properties": {
    "accessPolicies": [
      {
        "applicationId": null,
        "objectId": "29713030-308e-4392-b2d4-8d44d0db82b8",
        "permissions": {
          "certificates": [
            "Get",
            "List",
            "Update",
            "Create",
            "Import",
            "Delete",
            "Recover",
            "Backup",
            "Restore",
            "ManageContacts",
            "ManageIssuers",
            "GetIssuers",
            "ListIssuers",
            "SetIssuers",
            "DeleteIssuers"
          ],
          "keys": [
            "Get",
            "List",
            "Update",
            "Create",
            "Import",
            "Delete",
            "Recover",
            "Backup",
            "Restore"
          ],
          "secrets": [
            "Get",
            "List",
            "Set",
            "Delete",
            "Recover",
            "Backup",
            "Restore"
          ],
          "storage": null
        },
        "tenantId": "41a43e50-dc8c-46d7-9085-d04ba2e69216"
      },
      {
        "applicationId": null,
        "objectId": "4bd9e31a-8582-4d19-a7bb-748d7c7a384b",
        "permissions": {
          "certificates": null,
          "keys": null,
          "secrets": [
            "list",
            "get"
          ],
          "storage": null
        },
        "tenantId": "41a43e50-dc8c-46d7-9085-d04ba2e69216"
      }
    ],
    "createMode": null,
    "enablePurgeProtection": null,
    "enableRbacAuthorization": false,
    "enableSoftDelete": true,
    "enabledForDeployment": false,
    "enabledForDiskEncryption": false,
    "enabledForTemplateDeployment": false,
    "hsmPoolResourceId": null,
    "networkAcls": null,
    "privateEndpointConnections": null,
    "provisioningState": "Succeeded",
    "publicNetworkAccess": "Enabled",
    "sku": {
      "family": "A",
      "name": "Standard"
    },
    "softDeleteRetentionInDays": 90,
    "tenantId": "41a43e50-dc8c-46d7-9085-d04ba2e69216",
    "vaultUri": "https://azkv1226.vault.azure.net/"
  },
  "resourceGroup": "eastasia",
  "systemData": {
    "createdAt": null,
    "createdBy": null,
    "createdByType": null,
    "lastModifiedAt": "2021-12-26T07:43:15.148000+00:00",
    "lastModifiedBy": "lewisdba@yahoo.com.tw",
    "lastModifiedByType": "User"
  },
  "tags": {},
  "type": "Microsoft.KeyVault/vaults"
}

c:\python>

--安裝pip套件

f054517558d249a58cc2368f1141fa3f

'''